In [ ]:
# Y - wellness_satisfaction use dataV2_cohortV4_wideV2

In [ ]:

import pandas as pd
cols = pd.read_csv("https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/var_names.csv")
data = pd.read_csv("https://raw.githubusercontent.com/pointOfive/stat130chat130/main/CP/CSCS_data_anon.csv",
                   na_values=["9999", "", " ", "Presented but no response", "NA"])
empty = (data.isna().sum()==data.shape[0])
data = data[empty.index[~empty]] # keep non empty columns only
data.shape

In [ ]:
data.SURVEY_random_id.notna().value_counts()
dataV2 = data[data.REMOVE_case=='No'].copy()
dataV2.shape

In [ ]:
dataV2_cohort = dataV2[dataV2.SURVEY_cohort_participant].copy()
dataV2_cohort.shape

# And remove year 2023 for which there's not yet much data collected
dataV2_cohortV2 = dataV2_cohort[dataV2_cohort.SURVEY_collection_year!=2023].copy()
dataV2_cohortV2.shape

# And remove columns that have less than some number of missing values
# We could consider no missing values using 850, or...?
missingness_limit = 100 # this retains 166 of 1024 columns that aren't fully empty
columns2keep = dataV2_cohortV2.isna().sum() < missingness_limit
columns2keep = columns2keep.index[columns2keep]
dataV2_cohortV3 = dataV2_cohortV2[columns2keep].copy()
dataV2_cohortV3.shape

# The data itself then looks like this
pd.set_option('display.max_columns', dataV2_cohortV3.shape[1]) # Can cause jupyter notebooks to crash
# DO NOT USE in conjuection with pd.set_option('display.max_rows', 1000)
dataV2_cohortV3

In [ ]:
dataV2_cohortV3.groupby('SURVEY_collection_type').size()
SURVEY_collection_type

import plotly.express as px

fig = px.box(dataV2_cohortV3[dataV2_cohortV3.Secs_per_q<30],
             x='SURVEY_collection_type', y='Secs_per_q')
fig.update_layout(xaxis_title='SURVEY_YEAR', yaxis_title='Secs_per_q',
                  title='Boxplot of Secs_per_q by SURVEY_YEAR').show()

In [ ]:
dataV2_cohortV4 = dataV2_cohortV3[dataV2_cohortV3.Secs_per_q<30].copy()
dataV2_cohortV4.loc[:, 'SURVEY_YEAR'] = (dataV2_cohortV4.SURVEY_collection_type+" "+dataV2_cohortV4.SURVEY_collection_year.astype(str)).values
# dataV2_cohortV4.SURVEY_YEAR.value_counts()
# SURVEY_YEAR
# cohort 2022    446
# cross 2021     368

# we'll focus on some of the numeric outcomes in the data set... including Secs_per_q
dataV2_cohortV4_wide = \
dataV2_cohortV4.melt(id_vars=['UNIQUE_id','SURVEY_YEAR'],
                     value_vars=['Secs_per_q',
                                'LONELY_ucla_loneliness_scale_score',
                                'WELLNESS_life_satisfaction',
                                'WELLNESS_malach_pines_burnout_measure_score',
                                'LONELY_dejong_emotional_social_loneliness_scale_score',
                                'LONELY_dejong_emotional_loneliness_sub_scale_score',
                                'LONELY_dejong_social_loneliness_sub_scale_score',
                                'PSYCH_zimet_multidimensional_social_support_scale_score',
                                'PSYCH_zimet_multidimensional_social_support_family_subscale_score',
                                'PSYCH_zimet_multidimensional_social_support_significant_other_subscale_score',
                                'PSYCH_zimet_multidimensional_social_support_friends_subscale_score',
                                'WELLNESS_subjective_happiness_scale_score',
                                'WELLNESS_phq_score', 'WELLNESS_gad_score'])
# and we'll put these in columns for "cross 2021" and "cohort 2022"
dataV2_cohortV4_wide['variable'] = dataV2_cohortV4_wide['variable'] + " ("+dataV2_cohortV4_wide['SURVEY_YEAR']+")"
# on the basis of UNIQUE_id which links individuals across studies
dataV2_cohortV4_wide = dataV2_cohortV4_wide.pivot(index='UNIQUE_id', columns='variable', values='value')
# and consider fully observed data only
dataV2_cohortV4_wideV2 = dataV2_cohortV4_wide.dropna()
dataV2_cohortV4_wideV2

In [ ]:
# This could be used for the proportion of a paired differences test...
(dataV2_cohortV4_wideV2['Secs_per_q (cohort 2022)']>
 dataV2_cohortV4_wideV2['Secs_per_q (cross 2021)']).sum(), dataV2_cohortV4_wideV2.shape[0]

# This could be used for the proportion for a test of "random coin flipping"
col = 'Wellness_life_satisfaction'
(dataV2_cohortV4_wideV2[col]>1).sum(), dataV2_cohortV4_wideV2.shape[0]

dataV2_cohortV4_wideV2['Wellness_life_satisfaction (cohort 2022)'].value_counts()
# histogram of Wellness_life_satisfaction
px.histogram(dataV2_cohortV4_wideV2, x='Wellness_life_satisfaction (cohort 2022)')

In [ ]:
dataV2_cohortV4_wideV2.dropna(inplace=True)
#scatter plot
COVID_prevention_distancing，COVID_vaccinated，COVID_prevention_household，LONELY_change_pre_covid，
WELLNESS_self_rated_mental_health，WELLNESS_self_rated_physical_health，LONELY_ucla_loneliness_scale_score，
PSYCH_zimet_multidimensional_social_support_scale_score，CONNECTION_activities_community_pm
It is best that all variables are numerical
px.scatter(dataV2_cohortV4_wideV2, x='WELLNESS_self_rated_mental_health', y='Wellness_life_satisfaction')
#corplot correlation plot Look at the relationship between y and each x
px.imshow(dataV2_cohortV4_wideV2.corr())
#smf.ols(model)
#AIC, BIC, R square (0.6)

In [ ]:
#2021 compare 2022
# based on coefficient， If the coeff in 2022 is large (+) and the coeff in 2021 is small, people gradually accept take action to prevent during covid, 2022 after covid
mlr = smf.ols(formula='Wellness_life_satisfaction ~ COVID_prevention_distancing + COVID_vaccinated+COVID_prevention_household+LONELY_change_pre_covid+WELLNESS_self_rated_mental_health+WELLNESS_self_rated_physical_health+LONELY_ucla_loneliness_scale_score+PSYCH_zimet_multidimensional_social_support_scale_score+CONNECTION_activities_community_pm',data=dataV2_cohortV4_wideV2)
mlr_fit = mlr.fit() y= 20+3.2x, 20+x
mlr_fit.summary()

In [ ]:
decision tree make prediction，classification
# compare with mlr
# limitation（wrote on the slides）: mlr no multicollinearity , The |correlation| between xi should be small, corplot, in general we did not check this assumption
# normality: residual qqplot
# equal variance：qqplot
# no check linear regression assumption
# There may be a non-linear relationship between variables X and Y, but we have not learned other relationships, so this is also a limitation.

In [ ]:
Intro：research question+data summary+population+Why choose this topic+picture
Method：choose what model，what exactly do
Result：on the base of model，mlr need to write linear regresssion，and write 95%CI，parameter（R square， AIC， BIC）
Interpretation（conclusion samiliar）：interpret the result，and check weather have linear relationship（based on p value）if want to compare，explian the difference，pvalue，corrcoef
limitation: model limitation, research question limitation, graph limitation(graph display Some limitations came out)
12.2 due